In [37]:
# Dependencies
# ----------------------------------
import pandas as pd
from numpy import genfromtxt
from time import time
from datetime import datetime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, Session

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, Date

# Create Database

In [38]:
#Load csv file
birdData_file = "data/birdData.csv"

In [39]:
# read and display csv with Pandas
birdData_file_df = pd.read_csv(birdData_file, index_col=0)
birdData_file_df = birdData_file_df.rename(columns = {'City':'ID'})
birdData_file_df.head()

,ID,English Name,Genus,Location,Latitude,Longitude,Quality Score,Author Remarks,Recording Type,MP3 Download URL
0,92020,Canada Goose,Branta,"Sauvie Island, Multnomah Co., Oregon",45.6820,-122.8090,A,natural vocalization; calls from flock of 50 o...,call,//www.xeno-canto.org/92020/download
1,530562,Cackling Goose,Branta,"NW Reeder Rd, Canal, Access, Sauvie Island, Mu...",45.7020,-122.7810,no score,NaN,call,//www.xeno-canto.org/530562/download
2,530560,Cackling Goose,Branta,"NW Reeder Rd, Canal, Access, Sauvie Island, Mu...",45.7020,-122.7810,no score,NaN,call,//www.xeno-canto.org/530560/download
3,530316,Cackling Goose,Branta,"NW Oak Island Rd, Sauvie Island, Multnomah Cou...",45.6989,-122.8352,no score,NaN,call,//www.xeno-canto.org/530316/download
4,477242,Cackling Goose,Branta,"William Finley NWR, Bruce Rd Marsh, Willamette...",44.3897,-123.2960,no score,Recording amplified. High pass filter.,call,//www.xeno-canto.org/477242/download


In [40]:
birdMetadata_df = birdData_file_df[['ID', 'English Name', 'Genus', 'Location', 'Latitude', 'Longitude']]
birdMetadata_df

,ID,English Name,Genus,Location,Latitude,Longitude
0,92020,Canada Goose,Branta,"Sauvie Island, Multnomah Co., Oregon",45.6820,-122.8090
1,530562,Cackling Goose,Branta,"NW Reeder Rd, Canal, Access, Sauvie Island, Mu...",45.7020,-122.7810
2,530560,Cackling Goose,Branta,"NW Reeder Rd, Canal, Access, Sauvie Island, Mu...",45.7020,-122.7810
3,530316,Cackling Goose,Branta,"NW Oak Island Rd, Sauvie Island, Multnomah Cou...",45.6989,-122.8352
4,477242,Cackling Goose,Branta,"William Finley NWR, Bruce Rd Marsh, Willamette...",44.3897,-123.2960
...,...,...,...,...,...,...
495,534244,Purple Finch,Haemorhous,"William Finley NWR, Cabell Marsh Overlook, Ben...",44.4176,-123.3243
496,534243,Purple Finch,Haemorhous,"William Finley NWR, Cabell Marsh Overlook, Ben...",44.4176,-123.3243
497,371627,Purple Finch,Haemorhous,"Oregon (near Philomath), Benton County, Oregon",44.6055,-123.3900
498,436813,Cassin's Finch,Haemorhous,"Geneva, Jefferson County, Oregon",44.4707,-121.3932


In [41]:
birdRecordings_df = birdData_file_df[['ID','Recording Type', 'Quality Score', 'Author Remarks', 'MP3 Download URL']]
birdRecordings_df

,ID,Recording Type,Quality Score,Author Remarks,MP3 Download URL
0,92020,call,A,natural vocalization; calls from flock of 50 o...,//www.xeno-canto.org/92020/download
1,530562,call,no score,NaN,//www.xeno-canto.org/530562/download
2,530560,call,no score,NaN,//www.xeno-canto.org/530560/download
3,530316,call,no score,NaN,//www.xeno-canto.org/530316/download
4,477242,call,no score,Recording amplified. High pass filter.,//www.xeno-canto.org/477242/download
...,...,...,...,...,...
495,534244,"adult, female, song",no score,NaN,//www.xeno-canto.org/534244/download
496,534243,"adult, female, song",no score,NaN,//www.xeno-canto.org/534243/download
497,371627,"female, song",C,Female-plumaged bird singing from height of 4-...,//www.xeno-canto.org/371627/download
498,436813,song,no score,Recording amplified. High pass filter. Noise r...,//www.xeno-canto.org/436813/download


In [42]:
# Sets an object to utilize the default declarative base
Base = declarative_base()

In [43]:
# Creates Classes which will serve as the anchor points for our Tables
class birdMetadata(Base):
    __tablename__ = 'metadata'
    id = Column(Integer, primary_key=True)
    english_name = Column(String(40))
    genus = Column(String(20))
    location = Column(String(60))
    latitude = Column(Float)
    longitude = Column(Float)

In [44]:
class birdRecordings(Base):
    __tablename__ = 'recordings'
    id = Column(Integer, primary_key=True)
    type = Column(String(40))
    quality_score = Column(String(20))
    author_remarks = Column(String(100))
    download = Column(String(50))

In [45]:
if __name__ == "__main__":
    t = time()

    #Create the database
    engine = create_engine('sqlite:///data/birdDB.sqlite')
    Base.metadata.create_all(engine)

In [46]:
session = Session(bind=engine)

In [47]:
birdMetadata_df.to_sql("birdMetadata",
                      engine,
                      if_exists='replace',
                      index=False,
                      chunksize=500,
                      dtype={"id": Integer,
                             "english_name": String(40),
                             "genus": String(20),
                             "location": String(60),
                             "latitude": Float,
                             "longitude": Float
                            })

In [48]:
engine.execute("SELECT * FROM birdMetadata").fetchall()

[(92020, 'Canada Goose', 'Branta', 'Sauvie Island, Multnomah Co., Oregon', 45.681999999999995, -122.809),
 (530562, 'Cackling Goose', 'Branta', 'NW Reeder Rd, Canal, Access, Sauvie Island, Multnomah County, Oregon', 45.702, -122.781),
 (530560, 'Cackling Goose', 'Branta', 'NW Reeder Rd, Canal, Access, Sauvie Island, Multnomah County, Oregon', 45.702, -122.781),
 (530316, 'Cackling Goose', 'Branta', 'NW Oak Island Rd, Sauvie Island, Multnomah County, Oregon', 45.6989, -122.8352),
 (477242, 'Cackling Goose', 'Branta', 'William Finley NWR, Bruce Rd Marsh, Willamette Valley (near  Corvallis), Benton County, Oregon', 44.3897, -123.296),
 (477241, 'Cackling Goose', 'Branta', 'William Finley NWR, Bruce Rd Fields, Willamette Valley (near  Corvallis), Benton County, Oregon', 44.3895, -123.3159),
 (477240, 'Cackling Goose', 'Branta', 'William Finley NWR, Bruce Rd Marsh, Willamette Valley (near  Corvallis), Benton County, Oregon', 44.3897, -123.296),
 (477239, 'Cackling Goose', 'Branta', 'William

In [49]:
birdRecordings_df.to_sql("birdRecordings",
                          engine,
                          if_exists='replace',
                          index=False,
                          chunksize=500,
                          dtype={"id": Integer,
                                 "type": String(40),
                                 "quality_score": String(20),
                                 "author_remarks": String(100),
                                 "download": String(50)
                                  })

In [50]:
engine.execute("SELECT * FROM birdRecordings").fetchall()

[(92020, 'call', 'A', 'natural vocalization; calls from flock of 50 or so birds, some of which had red neck rings.', '//www.xeno-canto.org/92020/download'),
 (530562, 'call', 'no score', None, '//www.xeno-canto.org/530562/download'),
 (530560, 'call', 'no score', None, '//www.xeno-canto.org/530560/download'),
 (530316, 'call', 'no score', None, '//www.xeno-canto.org/530316/download'),
 (477242, 'call', 'no score', 'Recording amplified. High pass filter.', '//www.xeno-canto.org/477242/download'),
 (477241, 'call', 'no score', 'Recording amplified. High pass filter.', '//www.xeno-canto.org/477241/download'),
 (477240, 'call', 'no score', 'Recording amplified. High pass filter.', '//www.xeno-canto.org/477240/download'),
 (477239, 'call', 'no score', 'Recording amplified. High pass filter.', '//www.xeno-canto.org/477239/download'),
 (477238, 'call', 'no score', 'Amplified. High pass filter.', '//www.xeno-canto.org/477238/download'),
 (92019, 'call', 'A', 'natural vocalization; calls from a